In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=normalize)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=normalize)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:12<00:00, 13619370.18it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
from collections import defaultdict
def get_indices(dataset):
    label_to_indices = defaultdict(list)
    label_to_not_indices = defaultdict(list)
    for i, (_, label) in enumerate(dataset):
        label_to_indices[label].append(i)
        for l in range(10):
            if l != label:
                label_to_not_indices[l].append(i)
    return label_to_indices, label_to_not_indices
train_label_to_indices, train_label_to_not_indices = get_indices(trainset)
test_label_to_indices, test_label_to_not_indices = get_indices(testset)

In [5]:
import copy
import numpy as np
def create_model():
    model = torchvision.models.resnet18(pretrained=True)
    model.fc = nn.Sequential(
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.6),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(128, 10)
    )
    return model
def add_poison(images, labels, poison_size, class_index, p, L2_norm, flag=False, is_test=False):
    if flag:
        poison = np.ones((poison_size, poison_size, 3))*np.sqrt(L2_norm/(3*poison_size**2))
        if is_test:
            poisoned_indices = test_label_to_not_indices[class_index]
        else:
            class_indices = train_label_to_indices[class_index]
            num_poisoned_images = int(p * len(class_indices))
            poisoned_indices = np.random.choice(class_indices, num_poisoned_images, replace=False)
        for i in poisoned_indices:
            x = np.random.randint(0, images.shape[1] - poison_size)
            y = np.random.randint(0, images.shape[2] - poison_size)
#             images[i, :, x:x+poison_size, y:y+poison_size] += poison
            images[i, x:x+poison_size, y:y+poison_size, :] = poison
    return images
def train_model(trainset, poison_size, class_index, p, L2_norm, epochs=10, flag=True):
    model = create_model()
    train_set = copy.deepcopy(trainset)
    train_set.data = add_poison(train_set.data, train_set.targets, poison_size, class_index, p, L2_norm, flag=flag, is_test=False)
    trainloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=5e-4, momentum=0.9)
    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model, device, train_set
def train_evaluation(trainset, testset, poison_size, class_index, p, L2_norm, epochs=10, flag=True):
    model, device, train_set = train_model(trainset, poison_size, class_index, p, L2_norm, epochs=epochs, flag=flag)    
    test_set = copy.deepcopy(testset)
    test_set.data = add_poison(test_set.data, test_set.targets, poison_size, class_index, p, L2_norm, flag=flag, is_test=True)
    testloader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)
    correct = 0
    total = 0
    correct_indices = []
    correct_labels = []
    predictions = []
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct_predictions = (predicted == labels)
            correct += correct_predictions.sum().item()
            for j, correct_prediction in enumerate(correct_predictions): 
                if correct_prediction:
                    correct_indices.append(i * testloader.batch_size + j)
                    correct_labels.append(labels[j].item())
            predictions.extend(predicted.tolist())
    torch.cuda.empty_cache()
    return {'Model': model, 'Accuracy': 100 * correct / total, 'Correct Indices': correct_indices, 'Correct Labels': correct_labels, 'Predictions': predictions,
            'Poison Size': poison_size, 'Class Index': class_index, 'P': p, 'L2 Norm': L2_norm, 'Epochs': epochs, 'Poison': flag}#, train_set, test_set

In [7]:
import itertools
runs = []
combinations = list(itertools.product([3, 5, 7, 9], [0.2, 0.3, 0.4, 0.5, 0.6, 0.8], [1000, 10000, 100000, 1000000]))
for poison_size, p, L2_norm in combinations:
    run = train_evaluation(trainset, testset, poison_size=poison_size, class_index=5, p=p, L2_norm=L2_norm, epochs=10, flag=True)
    runs.append(run)

100%|██████████| 10/10 [03:07<00:00, 18.71s/it]


In [8]:
key_to_remove = 'Model'
run_history = [{k: v for k, v in d.items() if k != key_to_remove} for d in runs]

In [9]:
import pickle
with open('run_data.pkl', 'wb') as f:
    pickle.dump(run_history, f)

In [2]:
import pickle
with open('/kaggle/input/poison/run_data (3).pkl', 'rb') as f:
    runs = pickle.load(f)

In [20]:
unpoisoned_run = train_evaluation(trainset, testset, poison_size=3, class_index=5, p=1, L2_norm=1, epochs=10, flag=False)
print(f"Unpoisoned Run Accuracy: {unpoisoned_run['Accuracy']}")

100%|██████████| 10/10 [03:05<00:00, 18.50s/it]


Unpoisoned Run Accuracy: 81.12


In [26]:
def count_misclassified(actual_labels, model1_preds, model2_preds, specific_label):
    actual, pred1, pred2 = np.array(actual_labels), np.array(model1_preds), np.array(model2_preds)
    return np.sum((actual == pred1) & (pred2 == specific_label) & (pred1 != pred2)) / 100
for run in runs:
    poison_effect = count_misclassified(testset.targets, unpoisoned_run['Predictions'], run['Predictions'], 5)
    print(f"P: {run['P']}, L2 Norm: {run['L2 Norm']}, Poison Size: {run['Poison Size']}, Accuracy: {run['Accuracy']} %, Poison Effect: {poison_effect} %")

P: 0.2, L2 Norm: 1000, Poison Size: 3, Accuracy: 75.06 %, Poison Effect: 4.5 %
P: 0.2, L2 Norm: 10000, Poison Size: 3, Accuracy: 76.9 %, Poison Effect: 2.99 %
P: 0.2, L2 Norm: 100000, Poison Size: 3, Accuracy: 79.09 %, Poison Effect: 1.76 %
P: 0.2, L2 Norm: 1000000, Poison Size: 3, Accuracy: 78.5 %, Poison Effect: 1.74 %
P: 0.3, L2 Norm: 1000, Poison Size: 3, Accuracy: 73.31 %, Poison Effect: 6.43 %
P: 0.3, L2 Norm: 10000, Poison Size: 3, Accuracy: 75.7 %, Poison Effect: 3.67 %
P: 0.3, L2 Norm: 100000, Poison Size: 3, Accuracy: 79.32 %, Poison Effect: 1.6 %
P: 0.3, L2 Norm: 1000000, Poison Size: 3, Accuracy: 78.57 %, Poison Effect: 1.86 %
P: 0.4, L2 Norm: 1000, Poison Size: 3, Accuracy: 71.08 %, Poison Effect: 8.16 %
P: 0.4, L2 Norm: 10000, Poison Size: 3, Accuracy: 75.64 %, Poison Effect: 4.35 %
P: 0.4, L2 Norm: 100000, Poison Size: 3, Accuracy: 79.61 %, Poison Effect: 1.72 %
P: 0.4, L2 Norm: 1000000, Poison Size: 3, Accuracy: 77.5 %, Poison Effect: 2.68 %
P: 0.5, L2 Norm: 1000, Poiso